In [2]:
import numpy as np
import pandas as pd

In [5]:
s_cand = pd.read_pickle('../dataframes/galaxy_frames/s_candidates.pkl')
n_cand = pd.read_pickle('../dataframes/galaxy_frames/n_candidates.pkl')

In [19]:
for i in range(len(n_cand)):
    slrm_text= ["#!/bin/bash\n#SBATCH -J g_beam-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o gbeam{0}-%j.out\n\
#SBATCH -e gbeam{0}-%j.err\n\nchmod a+x C_beams_instruct.py\n\n\
python C_beams_instruct.py {1} 'south' {2} {3} &\npython C_beams_instruct.py {4} 'north' {5} {6} &\n\
    \nwait\n\nexit 0".format(i + 1, s_cand.gids[s_cand.index[i]],s_cand.ra[s_cand.index[i]],s_cand.dec[s_cand.index[i]],
                            n_cand.gids[n_cand.index[i]],n_cand.ra[n_cand.index[i]],n_cand.dec[n_cand.index[i]])]

    np.savetxt('../cluster_scripts/gen_beam_{0}.slrm'.format(i + 1), slrm_text, fmt='%s')

In [24]:
slrm_text= ["#!/bin/bash\n#SBATCH -J g_beam-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o gbeam{0}-%j.out\n\
#SBATCH -e gbeam{0}-%j.err\n\nchmod a+x C_beams_instruct.py\n\npython C_beams_instruct.py {1} 'south' {2} {3} &\n\nwait\n\nexit 0".format(17, s_cand.gids[s_cand.index[16]],s_cand.ra[s_cand.index[16]],s_cand.dec[s_cand.index[16]])]

np.savetxt('../cluster_scripts/gen_beam_{0}.slrm'.format(17), slrm_text, fmt='%s')

In [25]:
slrm_text= ["#!/bin/bash\n#SBATCH -J g_beam-{0}\n#SBATCH -p background\n#SBATCH --time=10:00:00\n#SBATCH --mem-per-cpu=8000\n#SBATCH -o gbeam{0}-%j.out\n\
#SBATCH -e gbeam{0}-%j.err\n\nchmod a+x C_beams_instruct.py\n\npython C_beams_instruct.py {1} 'south' {2} {3} &\n\nwait\n\nexit 0".format(18, s_cand.gids[s_cand.index[17]],s_cand.ra[s_cand.index[17]],s_cand.dec[s_cand.index[17]])]

np.savetxt('../cluster_scripts/gen_beam_{0}.slrm'.format(18), slrm_text, fmt='%s')